In [1]:
from getpass import getpass
import os
from dotenv import load_dotenv

# 載入環境變數
load_dotenv()
# 兩個 API 的密鑰
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPERDEV_API_KEY"] = os.getenv("SERPERDEV_API_KEY")
# 判斷是否存在，不存在可透過手動輸入
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
if "SERPERDEV_API_KEY" not in os.environ:
    os.environ["SERPERDEV_API_KEY"] = getpass("Enter Serper API key:")

In [2]:
# 用於建立文件
from haystack.dataclasses import Document

# 定義一個文件，內容描述了德國南部巴伐利亞州首府慕尼黑
documents = [
    Document(
        content="""慕尼黑是德國南部巴伐利亞州的首府，充滿了豐富的文化遺產和現代城市的優雅。位於伊薩爾河畔，慕尼黑以其華麗的建築而聞名，如馬麗恩廣場的慕尼黑新市政廳和寧芬堡宮的宏偉壯觀。這座城市是藝術愛好者的天堂，擁有世界級的博物館，如阿爾特·皮納科特克，收藏了許多著名藝術家的傑作。慕尼黑也因其熱鬧的啤酒花園而聞名，當地人和遊客聚集在這裡享受城市著名的啤酒和傳統巴伐利亞美食。每年的慕尼黑啤酒節是世界上最大的啤酒節，吸引了來自世界各地的數百萬遊客。除了豐富的文化和美食享受外，慕尼黑還擁有如英國花園這樣的美麗公園，為繁忙的都市提供了一個寧靜的避風港。遊客們為慕尼黑的熱情好客所打動，使得這裡成為了想要體驗古老魅力和現代魅力的旅行者必去的目的地。"""
    )
]

In [3]:
# 定義一個提示模板，指導 LLM 回答查詢
prompt_template = """
根據文件回答下列問題。
如果答案未包含在文件中，請回覆
'no_answer'

Query: {{query}}

Documents:
{% for document in documents %}
    {{document.content}}
{% endfor %}
"""

In [4]:
from haystack.components.builders.prompt_builder import PromptBuilder
# 使用定義的模板建立 PromptBuilder
prompt_builder = PromptBuilder(template=prompt_template)

In [5]:
from haystack.components.generators import OpenAIGenerator
# 初始化 OpenAIGenerator，使用 gpt-4-turbo 模型
llm = OpenAIGenerator(model="gpt-4-turbo")

In [6]:
# 定義一個提示模板，用於網絡搜索
prompt_for_websearch = """
根據從 Web 檢索到的文件，回答以下查詢。
你的答案應表明你的答案是透過網路搜尋產生的。

Query: {{query}}
Documents:
{% for document in documents %}
    {{document.content}}
{% endfor %}
"""

In [7]:
from haystack.components.websearch.serper_dev import SerperDevWebSearch

# 初始化網絡搜索組件
websearch = SerperDevWebSearch()

In [8]:
from haystack.components.builders.prompt_builder import PromptBuilder
# 使用網絡搜索提示模板建立 PromptBuilder
prompt_builder_for_websearch = PromptBuilder(template=prompt_for_websearch)

In [9]:
from haystack.components.generators import OpenAIGenerator
# 初始化 OpenAIGenerator，使用 gpt-4-turbo 模型
llm_for_websearch = OpenAIGenerator(model="gpt-4-turbo")

In [10]:
from haystack.components.routers import ConditionalRouter

# 定義路由規則
routes = [
    {
        # 如果 LLM 回覆中包含 'no_answer'
        "condition": "{{'no_answer' in replies[0]}}",
        # 將查詢作為輸出
        "output": "{{query}}",
        # 輸出名稱
        "output_name": "go_to_websearch",
        # 輸出類型
        "output_type": str
    },
    {
        # 如果 LLM 回覆中不包含 'no_answer'
        "condition": "{{'no_answer' not in replies[0]}}",
        # 將回覆的答案作為輸出
        "output": "{{replies[0]}}",
        # 輸出名稱
        "output_name": "answer",
        # 輸出類型
        "output_type": str
    }
]

# 初始化 ConditionalRouter
router = ConditionalRouter(routes)

In [ ]:
from haystack import Pipeline

# 建立管道
pipe = Pipeline()